<a href="https://colab.research.google.com/github/XTEP63/Time_Series/blob/main/Proyecto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![iteso](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

**Integrantes del equipo:**
- Francisco Javier Gonzalez Lazalde
- Maximiliano Aguayo Villanueva
- Esteban Javier Berumen Nieto
- Abdon Islas Leon

**Docente:** Daniel Nuño

**Fecha:** 2 Diciembre 2024

# **Índice de contenidos**
1. [Introducción](Introducción)
2. [Vector autoregression](#Vector-autoregression)
3. [Descarga de los datos](#Descarga-de-los-datos)
3. [Transfomraciones matemáticas](#Transfomraciones-matemáticas)
4. [Ajuste de los modelos](#Ajuste-de-los-modelos)
5. [Validación de resultados y elección del modelo](#Validación-de-resultados-y-elección-del-modelo)
6. [Pronosticos](#Pronosticos)
7. [Conclusión](#Conclusión)

## **Introducción**


En el análisis y pronóstico de series de tiempo, una de las aplicaciones más relevantes es la proyección del desempeño futuro de portafolios de inversión. Este proyecto se centra en explorar y aplicar modelos avanzados de series de tiempo para estimar los precios futuros de las acciones que conforman el índice de Precios y Cotizaciones (IPC). El objetivo principal es proporcionar un marco metodológico sólido para la construcción de un portafolio simulado y el pronóstico de su rendimiento a corto plazo.

El proyecto se estructura en varias etapas clave: recopilación de datos históricos de precios, limpieza y transformación de las series de tiempo, selección y ajuste de modelos avanzados, y validación de resultados. Finalmente, se utiliza una metodología jerárquica "bottoms-up" para consolidar los pronósticos a nivel sectorial y del portafolio total.

Entre los modelos considerados para este análisis se incluyen un modelo simple como línea base, ARIMA para capturar patrones de autocorrelación, ETS para modelar componentes de tendencia y estacionalidad, regresión dinámica para incorporar variables exógenas, y Vector Autoregression (VAR) para modelar relaciones entre múltiples series de tiempo. Este conjunto diverso de modelos permite una comparación exhaustiva de diferentes enfoques, facilitando la selección del más adecuado para cada acción basado en métricas de error cuantitativas.

El análisis culmina con un pronóstico a tres meses, proporcionando una visión anticipada del comportamiento del portafolio en función de las tendencias pasadas y el peso de cada acción en el índice. Los resultados obtenidos son fundamentales no solo para entender el comportamiento de las series de tiempo, sino también para la toma de decisiones estratégicas en la gestión de inversiones.

## **Vector autoregression - Teoría**

El modelo de **Vector Autoregression (VAR)** es una herramienta estadística que analiza y predice cómo varias variables cambian juntas a lo largo del tiempo. A diferencia de otros modelos más simples, el VAR considera cómo cada variable afecta a las demás y a sí misma en diferentes momentos.

### Conceptos básicos:

- **Estructura del modelo**: Cada variable se relaciona con sus valores anteriores y los de otras variables.
- **Igualdad**: Todas las variables se tratan de la misma manera, sin asumir que una causa a la otra.

Un ejemplo simple de VAR con dos variables (Y y X) se ve así:


\begin{aligned}
Y_t &= a_1 + b_{11}Y_{t-1} + b_{12}X_{t-1} + \epsilon_{1t} \\
X_t &= a_2 + b_{21}Y_{t-1} + b_{22}X_{t-1} + \epsilon_{2t}
\end{aligned}


Donde:

- $a_1$ y $a_2$ son puntos de inicio.
- $b_{ij}$ son números que muestran cómo los valores pasados afectan al presente.
- $ϵ_{1t}$ y $ϵ_{2t}$ representa errores o sorpresas en cada predicción.

### Ventajas:

Ayuda a entender cómo diferentes variables se influyen entre sí.
Permite analizar relaciones cambiantes entre múltiples series de datos en el tiempo.

### Desventajas:

Los datos deben tener ciertas características para que el modelo funcione bien.
Puede complicarse si se usan muchas variables o se mira muy atrás en el tiempo.

El VAR se usa mucho en economía y finanzas para entender cómo se relacionan cosas como las tasas de interés, la inflación y los precios de las acciones.

## **Librerias Implentadas**

In [20]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import boxcox, shapiro
from sklearn.preprocessing import PowerTransformer


## **Descarga de los datos**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

portafolio = pd.read_excel("/content/drive/MyDrive/ipc_componentes.xlsx")
portafolio.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,nombre,ticker,peso,ticker yahoo,segmento
0,Alfa SA A,ALFA A,0.0129,ALFAA.MX,Materials
1,Alsea SA,ALSEA *,0.0113,ALSEA.MX,Consumer Discretionary
2,America Movil SAB de CV B,AMX B,0.0908,AMXB.MX,Communication Services
3,"Arca Continental, SAB de CV",AC *,0.0262,AC.MX,Consumer Staples
4,"Banco del Bajio, S.A.",BBAJIO O,0.0155,BBAJIOO.MX,Financials


In [29]:
ticker = yf.Tickers(portafolio['ticker yahoo'].tolist())
data = ticker.history(start= '2015-01-01', end= '2023-12-31', interval='1mo')['Close']

data.head()

[*********************100%***********************]  35 of 35 completed


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,65.702629,24.685486,37.795330,11.671397,152.225342,NaN,34.200211,15.430018,11.531771,37.656048,...,38.958515,38.346149,33.040314,262.346039,130.751801,NaN,56.302013,88.730598,23.547871,23.494356
2015-02-01 00:00:00+00:00,70.285492,29.106770,43.193314,11.671397,159.254822,NaN,37.858757,16.780146,13.125944,44.382278,...,39.022579,39.573227,34.646240,281.710846,138.726135,NaN,57.645134,92.414070,23.328476,29.694588
2015-03-01 00:00:00+00:00,69.477180,27.677942,42.006325,11.124300,159.635437,NaN,38.791317,16.133081,12.536795,41.835011,...,42.517673,41.436153,31.766651,235.338257,126.094269,NaN,56.418179,91.677376,23.035959,31.042505
2015-04-01 00:00:00+00:00,69.892464,27.992460,43.400566,11.737049,168.668884,NaN,36.944115,18.360481,13.335611,41.735298,...,45.450420,42.735741,35.035809,230.718735,136.695755,NaN,63.874294,101.554527,23.328476,29.519295
2015-05-01 00:00:00+00:00,70.519684,27.219727,43.626663,11.744344,176.417725,NaN,37.374535,16.991688,12.993212,42.756989,...,46.048355,40.197922,37.204605,242.054337,130.000076,NaN,61.688995,105.901924,22.910900,31.085051


## **Pesos y Segmentos**

In [23]:

print(f"{'Ticker':<25} {'Ticker Yahoo':<15} {'Segmento':<30} {'Peso':<10}")
print("=" * 85)


for index, row in portafolio.iterrows():
    print(f"{row['ticker']:<25} {row['ticker yahoo']:<15} {row['segmento']:<30} {row['peso']:<10.4f}")


Ticker                    Ticker Yahoo    Segmento                       Peso      
ALFA A                    ALFAA.MX        Materials                      0.0129    
ALSEA *                   ALSEA.MX        Consumer Discretionary         0.0113    
AMX B                     AMXB.MX         Communication Services         0.0908    
AC *                      AC.MX           Consumer Staples               0.0262    
BBAJIO O                  BBAJIOO.MX      Financials                     0.0155    
CUERVO *                  CUERVO.MX       Materials                      0.0051    
BOLSA A                   BOLSAA.MX       Financials                     0.0049    
CEMEX CPO                 CEMEXCPO.MX     Materials                      0.0649    
KOF UBL                   KOFUBL.MX       Consumer Staples               0.0217    
VESTA *                   VESTA.MX        Industrials                    0.0164    
LIVEPOL C-1               LIVEPOLC-1.MX   Consumer Discretionary         0.0

## **Train/Test**

In [24]:
train = data.loc[:'2022-12-01']
test = data.loc['2023-01-01':]
horizon = len(test)
print(f'Longitud de Train{train.shape}')
print(f'Longitud de Test{test.shape}')

Longitud de Train(96, 35)
Longitud de Test(12, 35)


In [25]:
train.tail()

Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-01 00:00:00+00:00,125.653046,11.681558,35.960003,15.888946,393.241547,31.970018,64.348747,32.173889,7.468695,56.374062,...,37.337002,114.007996,37.010098,165.589996,124.870323,81.147408,100.521362,24.065973,36.832081,60.962524
2022-09-01 00:00:00+00:00,132.854187,11.663333,35.556732,16.371164,364.940277,42.173584,68.770172,29.883965,6.958784,58.593662,...,35.852356,109.637718,32.999535,194.630005,120.814079,76.467598,104.911148,20.596834,37.288513,65.490280
2022-10-01 00:00:00+00:00,148.291687,11.991364,37.435387,17.357035,428.500702,46.135365,74.639496,31.758345,7.718651,78.658508,...,36.470963,136.733429,32.573822,222.009995,133.813980,71.484375,127.627121,19.904903,42.835155,70.869865
2022-11-01 00:00:00+00:00,147.137314,12.301172,38.163239,18.579897,440.969482,51.850597,79.556511,33.676933,8.818459,81.899521,...,43.302086,144.946106,36.942410,257.119995,153.584412,74.683937,130.187042,20.141865,42.755779,70.601349
2022-12-01 00:00:00+00:00,147.468277,11.326189,36.176388,17.252762,418.400360,50.767101,80.576439,33.358639,7.918616,81.536133,...,45.838348,129.654449,33.766144,238.179993,142.352432,79.271805,130.307175,16.776989,45.821808,64.403969


In [26]:
test.head()

Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00+00:00,154.989136,12.419627,44.281162,18.105921,470.210754,62.561478,91.532082,36.152527,10.018250,91.102051,...,52.324825,148.883652,37.304501,266.190002,175.657791,95.637360,150.583359,21.838524,50.862404,68.770508
2023-02-01 00:00:00+00:00,144.603638,10.934375,39.835339,17.641422,482.771729,57.822227,85.403191,31.537308,9.098411,91.337761,...,41.561165,151.393341,37.791691,218.910004,170.572922,104.638412,140.288177,17.326744,52.578987,67.414726
2023-03-01 00:00:00+00:00,152.451096,10.433216,42.402508,18.485102,507.967377,54.216091,88.683037,34.065952,9.918267,101.159027,...,40.739307,177.926193,38.964912,268.200012,171.815460,105.778496,127.211227,18.094505,56.260212,67.321228
2023-04-01 00:00:00+00:00,160.261185,10.479136,47.635197,18.485102,474.671173,48.897865,94.479050,35.029667,10.808112,104.400040,...,43.231388,174.389587,41.162228,278.000000,178.840622,109.768738,121.647774,17.345699,56.359432,67.760681
2023-05-01 00:00:00+00:00,170.277557,10.811080,50.605637,18.485102,457.391785,46.019573,92.853813,32.881203,10.618145,90.967812,...,36.320728,161.719727,37.203987,269.970001,164.580017,109.915848,127.349861,16.075577,55.657761,63.646519


## **Transfomraciones matemáticas**

In [27]:
def aplicar_boxcox(dataframe):
    resultado = dataframe.copy()
    lambdas = {}

    for columna in resultado.columns:
        serie = resultado[columna].dropna()

        if (serie <= 0).any():
            print(f"La columna {columna} tiene valores no positivos y no puede transformarse con Box-Cox.")
            continue

        stat, p_value = shapiro(serie)
        if p_value < 0.05:
            transformed_data, lambda_val = boxcox(serie)
            resultado.loc[serie.index, columna] = transformed_data
            lambdas[columna] = lambda_val
        else:
            lambdas[columna] = None

    return resultado, lambdas

In [30]:
data_boxcox, lambdas = aplicar_boxcox(data)

print("DataFrame transformado:")
display(data_boxcox)
print("\nLambdas aplicadas:")
print(lambdas)

DataFrame transformado:


Ticker,AC.MX,ALFAA.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,...,MEGACPO.MX,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,0.632167,3.272968,122.749521,1.473715,3.659535,NaN,0.555395,24.433214,6.907535,0.833469,...,38.958515,1.934555,3.296953,2.316007,130.751801,NaN,4.541748,13.503229,0.738084,4.779689
2015-02-01 00:00:00+00:00,0.632253,3.444812,148.577633,1.473715,3.682525,NaN,0.555557,27.258511,7.738654,0.835507,...,39.022579,1.941993,3.339037,2.324486,138.726135,NaN,4.571564,13.780053,0.737946,5.305291
2015-03-01 00:00:00+00:00,0.632239,3.392273,142.774069,1.457991,3.683737,NaN,0.555592,25.896603,7.434729,0.834819,...,42.517673,1.952691,3.262043,2.302622,126.094269,NaN,4.544353,13.725202,0.737756,5.408389
2015-04-01 00:00:00+00:00,0.632246,3.404064,149.597946,1.475530,3.711573,NaN,0.555521,30.642926,7.845956,0.834790,...,45.450420,1.959766,3.348942,2.300120,136.695755,NaN,4.701815,14.440699,0.737946,5.291626
2015-05-01 00:00:00+00:00,0.632257,3.374855,150.713384,1.475731,3.734136,NaN,0.555538,27.706817,7.670495,0.835078,...,46.048355,1.945657,3.402082,2.306142,130.000076,NaN,4.657540,14.742933,0.737673,5.411588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 00:00:00+00:00,0.632806,2.461091,233.262171,1.614236,4.161202,3.733799,0.556169,62.596538,7.974352,0.841129,...,39.013447,2.213298,3.415958,2.305753,157.880508,9.664302,5.513896,5.040875,0.746158,7.239871
2023-09-01 00:00:00+00:00,0.632790,2.467467,252.168404,1.542866,4.124030,3.749849,0.556169,59.113506,6.809879,0.841417,...,36.394905,2.207773,3.373731,2.284887,150.550308,9.587930,5.506942,3.971559,0.745759,7.179673
2023-10-01 00:00:00+00:00,0.632797,2.452835,231.164099,1.540129,4.083060,3.754626,0.556114,48.721503,6.509749,0.841538,...,33.417137,2.165060,3.185949,2.281167,144.053970,10.035603,5.625192,3.327076,0.745693,7.129600



Lambdas aplicadas:
{'AC.MX': -1.5797334781187926, 'ALFAA.MX': 0.012809894472050698, 'ALSEA.MX': 1.4230074365092635, 'AMXB.MX': -0.4587476892323569, 'ASURB.MX': -0.13368007815672064, 'BBAJIOO.MX': -0.017534654805201617, 'BIMBOA.MX': -1.7973726598274766, 'BOLSAA.MX': 1.2657200006507823, 'CEMEXCPO.MX': 0.7406129920534447, 'CHDRAUIB.MX': -1.1834287691481171, 'CUERVO.MX': 0.33388395355426986, 'FEMSAUBD.MX': -0.009234197911180724, 'GAPB.MX': 0.04563290568871829, 'GCARSOA1.MX': -0.9161846187846102, 'GCC.MX': 0.8592711953880243, 'GENTERA.MX': None, 'GFINBURO.MX': None, 'GFNORTEO.MX': -0.5553968135377377, 'GMEXICOB.MX': -0.5279213710351935, 'GRUMAB.MX': -0.1312343289477711, 'KIMBERA.MX': -0.3366932708008057, 'KOFUBL.MX': -0.8464404727649663, 'LABB.MX': 1.718813129563253, 'LACOMERUBC.MX': 0.15577016253735407, 'LIVEPOLC-1.MX': 0.34453619096513793, 'MEGACPO.MX': None, 'OMAB.MX': -0.39408800085005896, 'ORBIA.MX': -0.03414168724966447, 'PE&OLES.MX': -0.37967266356664503, 'PINFRA.MX': None, 'Q.MX': 

## **Formato y Elimanación de Na**

In [6]:
data.index = data.index.strftime('%Y-%m')
data.index = pd.to_datetime(data.index)
data = data.melt(ignore_index=False)
df = pd.DataFrame({"ds": data.index,
                   "y": data['value'],
                   "unique_id": data['Ticker']})
df.reset_index(drop=True, inplace=True)
df.dropna(inplace=True)
df.head()

,ds,y,unique_id
0,2015-01-01,65.702621,AC.MX
1,2015-02-01,70.285492,AC.MX
2,2015-03-01,69.477180,AC.MX
3,2015-04-01,69.892456,AC.MX
4,2015-05-01,70.519676,AC.MX


## **Ajuste de los modelos**

## **Validación de resultados y elección del modelo**

## **Pronosticos**

## **Conclusión**